In [2]:
import json
DATA_PATH = '../data/'


In [4]:
fname = "pretty_scopus_hazard_map_chile_0-25"
with open(f"{DATA_PATH}/{fname}.json", "r") as f:
    data = json.load(f)

In [10]:
for entry in data['search-results']['entry']:
    print(entry)
    break

{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85177228684'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85177228684?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85177228684&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85177228684&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85177228684', 'dc:identifier': 'SCOPUS_ID:85177228684', 'eid': '2-s2.0-85177228684', 'dc:title': 'SIGeR-RV: A Web-Geographic Information System-Based System for Risk Management of Road Networks Exposed to Natural Hazards', 'dc:creator': 'Chamorro A.', 'prism:publicationName': 'Transportation Research Record', 'prism:issn': '03611981', 'prism:eIssn': '21694052',

In [60]:
out = {
    'types': {
        'Article': {
            'pluralLabel': 'Articles',
            "uri": "http://simile.mit.edu/2007/05/article-ns#Article"
        },
        'Affiliation': {
            'pluralLabel': 'Affiliations',
            "uri": "http://simile.mit.edu/2007/05/article-ns#Affiliation"
        }
    },
    'properties': {
        'doi': {
            'label': 'DOI',
            "uri": "http://simile.mit.edu/2007/05/article-ns#doi"
        },
        'title': {
            'label': 'Title',
            "uri": "http://simile.mit.edu/2007/05/article-ns#title"
        },
        'pubName': {
            'label': 'Publication Name',
            "uri": "http://simile.mit.edu/2007/05/article-ns#title"
        },
        'pubDate': {
            'label': 'Publication Date',
            "uri": "http://simile.mit.edu/2007/05/article-ns#pubDate"
        },
        'pubType': {
            'label': 'Publication Type',
            "uri": "http://simile.mit.edu/2007/05/article-ns#pubType"
        },
        'mainAuthor': {
            'label': 'Principal Author',
            "uri": "http://simile.mit.edu/2007/05/article-ns#mainAuthor"
        },
        'year': {
            'label': 'Year',
            "uri": "http://simile.mit.edu/2007/05/article-ns#year"
        },
        'country': {
            'label': 'Country',
            "uri": "http://simile.mit.edu/2007/05/article-ns#country"
        },
        'city': {
            'label': 'City',
            "uri": "http://simile.mit.edu/2007/05/article-ns#city"
        },
        'link': {
            'label': 'Link',
            "uri": "http://simile.mit.edu/2007/05/article-ns#link"
        },
        'affiliation': {
            'label': 'Affiliation',
            "uri": "http://simile.mit.edu/2007/05/article-ns#affiliation",
            'valueType': 'item'
        },
    }
}

props = {
    'title': 'dc:title', 
    'label': 'dc:title', 
    'pubName': 'prism:publicationName', 
    'pubDate': 'prism:coverDate',
    'doi': 'prism:doi',
    'pubType': 'prism:aggregationType',
    'mainAuthor': 'dc:creator'
}
out['items'] = []
allaffs = {}
for ri, entry in enumerate(data['search-results']['entry']):
    out_entry = {i:entry.get(j, "") for i, j in props.items() if j in entry}
    links = {link['@ref']: link['@href'] for link in entry['link']}
    out_entry.update({'link': links.get('scopus', '')})
    out_entry.update({'type': 'Article'})
    if 'pubDate' in out_entry:
        out_entry.update({'year': out_entry['pubDate'][:4]})
    affs = []
    for affiliation in entry.get('affiliation', []):
        affs.append(affiliation['affilname'])
        allaffs[affiliation['affilname']] = {
            'label': affiliation['affilname'],
            'type': 'Affiliation',
            'country': affiliation['affiliation-country'],
            'city': affiliation['affiliation-city']
        }
    out_entry.update({'affiliation': affs})
    out_entry.update({'uri': f"http://simile.mit.edu/2007/05/article-ns/Article/{ri}"})
    out['items'].append(out_entry)
for ri, entry in enumerate(allaffs):
    allaffs[entry].update({'uri': f"http://simile.mit.edu/2007/05/article-ns/Affiliation/{ri}"})
out['items'].extend(list(allaffs.values()))

with open(f"../items.json", "w") as f:
    json.dump(out, f, indent=2)